In [1]:
import cv2
import imutils
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from imutils.perspective import four_point_transform
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from skimage.segmentation import clear_border
from sudoku import Sudoku

C:\Users\garel\miniconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\garel\miniconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\garel\miniconda3\envs\tensorflow_env\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\garel\miniconda3\envs\tensorflow_e

In [2]:
class sudoku_solver:
        
    tf.logging.set_verbosity(tf.logging.ERROR)
    
    def __init__(self,path):
        '''ingresar ruta de la imagen que se quiere analizar'''
        self.img= cv2.imread(path)
        
    def get_board(self):
        '''en base a la imagen ingresada previamente devuelve la cuadricula del sudoku unicamente'''
        img_gray = cv2.cvtColor(self.img, cv2.COLOR_RGB2GRAY)
        th = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11,2)
        thresh = cv2.bitwise_not(th)
        
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = imutils.grab_contours(cnts)
        cnt = sorted(cnt, key=cv2.contourArea, reverse= True)[:5]
        
        for c in cnt:
            perimetro = cv2.arcLength(c, True)
            epsilon = 0.05 * perimetro
            self.aprox = cv2.approxPolyDP(c, epsilon,True)
            if len(self.aprox) == 4: break
                
        self.sudoku_board = four_point_transform(self.img, self.aprox.reshape(4,2))
        return self.sudoku_board
        
    def img_pipeline(self,single_number, th):
        '''para uso interno, toma una caudricula unica del sudoku a la vez y un nivel de threshold, limpia la imagen y calcula
        las probabilidades segun una red neuronal de que sea determinado numero, si las probabilidades son altas
        devuelve dicho numero, en caso contrario devuelve 0'''
        cnts,thresh = self.grab_cnts(single_number,th)
        try:
            c = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(c) > 50:
                digit = self.mask(thresh, c)
                prob =  self.predictor(digit)
                if prob[0][prob.argmax(axis=-1)[0]]>0.98:
                    return prob.argmax(axis=-1)[0]
                else:
                    th = th-5
                    num = self.img_pipeline(single_number,th)
                    return num
            else:
                return 0
        except:
            return 0

    def predictor(self,digit):
        '''para uso interno, toma la imagen de un numero en un canal y devuelve 10 probabilidades de que se corresponda con
        los numeros del 0 al 9'''
        number = cv2.resize(digit,(28,28))
        number = number.astype("float") / 255.0
        number = img_to_array(number)
        number = np.expand_dims(number, axis=0)
        prob = self.clasificador.predict(number)
        return prob
        
    def grab_cnts(self,single_cell, th):
        '''para uso interno, en base a una imagen de una casilla unica la limpia y la devuelve en 1 canal junto con sus
        contornos'''
        gray = cv2.cvtColor(single_cell, cv2.COLOR_RGB2GRAY)
        thresh = cv2.threshold(gray, th, 240, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
        thresh = clear_border(thresh)
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        return cnts, thresh
        
    def mask(self,thresh, c):
        '''para uso interno, en base a una imagen en 1 canal, dibuja el mayor contorno sobre una mascara para eliminar
        ruido'''
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 200, -1)
        digit = cv2.bitwise_and(thresh, thresh, mask=mask)
        return digit
    
    def posible(self,x,y,n):
        '''para uso interno, en base a una posicion y el numero en dicha posicion devuelve True si no hay error en su 
        ubicacion y false en caso que exista error en la asignacion del numero'''
        if n == 0:
            return True
        for i in range(0,9):
            if self.matriz[x][i] == n and i != y:
                return False
        for i in range(0,9):
            if self.matriz[i][y] == n and i != x:
                return False

        x0 = (x//3)*3
        y0 = (y//3)*3

        for i in range(0,3):
            for j in range(0,3):
                if self.matriz[x0+i][y0+j] == n:
                    if x0+i == x and y0+j == y:
                        pass
                    else:
                        return False
        return True
        
    def get_matrix(self,model_path,th=155):
        '''toma la ruta del modelo de red que se va a usar para analizar los numero y un nivel de threshold inical
        devuelve la matriz ya procesada'''
        self.hight= int(self.sudoku_board.shape[0]/9)
        self.widht = int(self.sudoku_board.shape[1]/9)
        self.clasificador = load_model(model_path)
        self.matriz = np.zeros((9,9),dtype=int)
        for i in range(9):
            for j in range(9):
                single_number = self.sudoku_board[self.hight*i:self.hight*(i+1),self.widht*j:self.widht*(j+1)]
                number = self.img_pipeline(single_number,th)
                self.matriz[i][j] = number
        return self.matriz
    
    def chequeo(self):
        '''no toma ningun argumento, en caso de tener algun numero colocado de forma incorrecta y con poca probabilidad de
        esta en dicho lugar lo reemplaza por 0, de esta forma el sudoku va a tener mas soluciones posibles pero no va a ser
        erroneo'''
        for i in range(9):
            for j in range(9):
                if not self.posible(i,j,self.matriz[i][j]):
                    single_number = self.sudoku_board[self.hight*i:self.hight*(i+1), self.widht*j:self.widht*(j+1)]
                    cnts,thresh = self.grab_cnts(single_number,130)
                    c = max(cnts, key=cv2.contourArea)
                    digit = self.mask(thresh, c)
                    prob =  self.predictor(digit)
                    if 85 >prob[0][prob.argmax(axis=-1)[0]]< 0.98:
                        self.matriz[i][j] =0
        return self.matriz
    
    def get_matrix_chequeo(self,model_path,th=155):
        self.matriz = self.get_matrix(model_path,th)
        self.matriz = self.chequeo()
        return self.matriz
    
    def solve(self):
        '''resuelve el sudoku y devuelve la matriz resuelta'''
        puzzle = Sudoku(3, 3, board=self.matriz.tolist())
        self.solved = puzzle.solve().board
        return self.solved
    
    def get_matrix_chequeo_solve(self,model_path,th=155):
        self.matriz = self.get_matrix(model_path,th)
        self.matriz = self.chequeo()
        self.solved = self.solve()
        return
    
    def dibujar_numeros(self):
        font        = cv2.FONT_HERSHEY_SIMPLEX
        fontScale   = 1.5
        fontColor1  = (255,255,0)
        fontColor2  = (255,0,255)
        lineType    = 2
        x,y = int(solver.widht/2)-15, int(solver.hight-10)

        for i in range(9):
            for j in range(9):
                pos = (x+self.widht*j, y+self.hight*i)
                num1 = self.matriz[i][j]
                num2 = self.solved[i][j]
                if num1 == 0:
                    cv2.putText(self.sudoku_board,f'{num2}',pos,font,fontScale,fontColor2,lineType)
                else:
                    cv2.putText(self.sudoku_board,f'{num1}',pos,font,fontScale,fontColor1,lineType)
    